In [15]:
import os
import json

import openai

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## OpenAI Function Calling

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    """
    Get current weather in a given location
    """
    weather_info = {
        "location": location, 
        "temperature": "72", 
        "unit": unit, 
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

In [3]:
# Define a function
functions = [
    {
        "name": "get_current_weather", 
        "description": "Get the current weather in in a given location", 
        "parameters": {
            "type": "object", 
            "properties": {
                "location": {
                    "type": "string", 
                    "description": "The city and state, e.g. San Francisco, CA"
                }, 
                "unit": {"type": "string", "enum": ["celsius", "fahreheit"]}
            }, 
            "required": ["location"]
        }
    }
]

In [4]:
messages = [
    {
        "role": "user", 
        "content": "What's the weather like in Boston?"
    }
]

In [5]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613", 
    messages=messages, 
    functions=functions
)

In [6]:
print(response)

ChatCompletion(id='chatcmpl-90lKr8hriXVnMGUzvT24omCA0hQym', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1709968985, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=85, total_tokens=103))


## LangChain Expression Language(LCEL)

- LCEL include chain of components.
- Can be linked in Linux Pipe style.
- Components implement "Runnable" protocol.
- Common methods include:
    - invoke
    - stream
    - batch
- Common properties
    - input_schema
    - output schema
- Common i/o
- Component: Prompt
    - Input type: Dictionary
    - Output type: Prompt Value
- Component: Retriever
    - Input type: Single String
    - Output type: List of Documents
- Component: LLM
    - Input type: String, List of messages or Prompt value
    - Output type: String
- Component: ChatModel
    - Input type: String, List of messages or Prompt Value
    - Output type: ChatMessage
- Component: Tool
    - Input type: String/Dictionary
    - Output type: Tool dependent
- Component: Output Parser
    - input type: Output of LLM or ChatModel
    - output type: Parser dependent
- Runnables Support
    - Async, Batch and streaming support
    - Fallbacks
    - Parallelism
        - LLM calls can be time consuming
        - Any components that can be run in parallel are
    - Logging is built in

In [12]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [13]:
chain = prompt | model | output_parser

In [14]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

### Retrieval Example

In [17]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"], 
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.